In [ ]:
import torch
import os
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import csv
import json
import random
torch.random.manual_seed(0)

In [ ]:
dir_path = "/kaggle/input/statistics/DataModel/ecom_tables_and_columns_new_with_dimensions"
files = [dir_path + "/" + f for f in os.listdir(dir_path) if os.path.isfile(os.path.join(dir_path,f))]
files = sorted(files)
dir_path = "/kaggle/input/statistics/DataModel/metrics_table_new"
k = []
for f in os.listdir(dir_path):
    if os.path.isfile(os.path.join(dir_path,f)):
        k.append(dir_path + "/" + f)
k = sorted(k)
for i in k:
    files.append(i)
files.append("/kaggle/input/statistics/DataModel/relationships_minimized.json")
data = []
for file in files:
    with open(file, 'r') as reader:
        d = json.load(reader)
    data.append(str(d))
input_quests = []
with open('/kaggle/input/statistics/DataModel/inputQuestions.csv') as file_obj:  
    reader_obj = csv.reader(file_obj) 
    for row in reader_obj: 
        input_quests.append(str(row)[2:-2])
    

In [ ]:
messages = []
for i in range(len(data)):
    question = random.choice(input_quests[5*i:5*i+1])
    messages.append({"role": "user", "content": question})
    messages.append({"role": "assistant", "content": data[i]})
    
questions = []
with open('/kaggle/input/statistics/DataModel/Questions.csv') as file_obj:  
    reader_obj = csv.reader(file_obj) 
    for row in reader_obj: 
        questions.append(str(row)[2:-2])


In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-128k-instruct", 
    device_map="cuda", 
    torch_dtype="auto", 
    trust_remote_code=True, 
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-128k-instruct")

In [ ]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

generation_args = {
    "max_new_tokens": 2048,
    "return_full_text": False,
    "temperature": 0.0,
    "do_sample": False,
}

In [ ]:
#Generate output using previous messages
trained_model = pipe(messages, **generation_args)
#generate output for each question
for question in questions:
    query = [{"role": "user", "content": question}]
    output = pipe(query, **generation_args)
    print(output[0]['generated_text'])